---
author: Zeel B Patel
badges: true
categories:
- Data
date: '2024-12-27'
description: Download CPCB CAAQM locations using Selenium
title: Download CPCB CAAQM locations
toc: true
---

In [1]:
try:
    import selenium
except ModuleNotFoundError:
    %pip install selenium

import os
import re
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm, trange
from time import sleep, time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

!rm log.txt

def print_it(*args, **kwargs):
    print(*args, **kwargs)
    with open('log.txt', 'a') as f:
        print(*args, **kwargs, file=f)

global_init = time()

rm: cannot remove 'log.txt': No such file or directory


In [2]:
# Set up WebDriver
op = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=op)

# Navigate to the website and manually solve the CAPTCHA
driver.get("https://airquality.cpcb.gov.in/ccr/#/caaqm-dashboard-all/caaqm-landing")

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x556d1853ab9a <unknown>
#1 0x556d18220670 <unknown>
#2 0x556d18258971 <unknown>
#3 0x556d18254315 <unknown>
#4 0x556d182a0e66 <unknown>
#5 0x556d182a04b6 <unknown>
#6 0x556d18294923 <unknown>
#7 0x556d182626e7 <unknown>
#8 0x556d182636de <unknown>
#9 0x556d1850466b <unknown>
#10 0x556d18508611 <unknown>
#11 0x556d184f04e5 <unknown>
#12 0x556d18509192 <unknown>
#13 0x556d184d56ef <unknown>
#14 0x556d185299d8 <unknown>
#15 0x556d18529ba7 <unknown>
#16 0x556d185399ec <unknown>
#17 0x7fc996c7c609 start_thread


> Manually solve captcha before moving on to the next cell..

In [3]:
# leaflet-marker-icon custom-div-icon map_markers station_status_live leaflet-zoom-animated leaflet-interactive
all_station_markers = driver.find_elements(By.CLASS_NAME, 'leaflet-marker-icon')

all_stations_len = len(all_station_markers)
print("Total stations: ", all_stations_len)

Total stations:  549


In [4]:
def get_after(string, phrase):
    return string[string.index(phrase) + len(phrase):]

data = {}
all_station_markers = driver.find_elements(By.CLASS_NAME, 'leaflet-marker-icon')
marker_id = 0
progress_bar = tqdm(total=all_stations_len, desc="Progress")
while marker_id < all_stations_len:
    try:
        marker = all_station_markers[marker_id]
        driver.execute_script("arguments[0].click();", marker)
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'close')))
        children = driver.find_elements(By.CLASS_NAME, "col-md-12")
        assert "Station Name" in children[3].text
        
        # parse it
    
        station, address, location = children[3].text.split('\n')
        station = get_after(station, "Station Name: ")
        address = get_after(address, "Address: ")
        latitude, longitude = location.split(",")
        latitude = get_after(latitude, "Latitude: ")
        longitude = get_after(longitude, "Longitude: ")
        
        data[station] = {"address": address, "latitude": float(latitude), "longitude": float(longitude)}
        close = driver.find_element(By.CLASS_NAME, "close")
        close.click()
        sleep(0.5)
        marker_id += 1
        progress_bar.update(1)
    except Exception as e:
        driver.refresh()
        input("Please manually solve the Captcha")
        all_station_markers = driver.find_elements(By.CLASS_NAME, 'leaflet-marker-icon')

Progress:   0%|          | 0/549 [00:00<?, ?it/s]

In [8]:
df = pd.DataFrame(data).T
df.index.name = "station"
df.head(2)

,address,latitude,longitude
station,,,
"SIDCO Kurichi, Coimbatore - TNPCB","SIDCO Kurichi, Coimbatore, Tamil Nadu.",10.942451,76.978996
"Muradpur, Patna - BSPCB","S K Memorial Hall Premises, Near Gandhi Maidan...",25.619651,85.147382


In [9]:
df.to_csv("station_data.csv")